<div style="text-align: center;">
    <h1>Helpmate AI using Qdrant DB</h1>
</div>

# Problem statement

## Project goal
The life insurance policy documents are very elaborate and lengthy. It's a challenge to go through the entire document at once and understand it well.
This project provides an easy way to understand the document by asking asking questions and queries to the RAG based application.

## Project setup
- Install poetry for dependency management
- Install pyenv for python version management
- Install docker and docker-compose to start qdrant db locally
- run `docker-compose up -d` to start qdrant db, then view the qdrant dashboard at http://localhost:6333:dashboard
- run `poetry install` to install the dependencies
- run `poetry shell` to activate the virtual environment
- run `jupyter notebook` to start the jupyter notebook server

<div style="page-break-after: always;"></div>

# Solution Architecture

## Architecture diagram

![architecture.drawio.png](./architecture.drawio.png)

<div style="page-break-after: always;"></div>

## Architecture description & Design choices

### Vector Embedding Pipeline

1. **Read from disk**  
   Load the PDF policy document.

2. **Filter useful pages**  
   Retain only pages with meaningful content (those which are within a part and section).

3. **Read pages**  
   Extract text content using a PDF parser (using `pdfplumber`).

4. **Chunk by part/section**  
   Split document hierarchically to maintain structure.

5. **Chunk by part/section/article**  
   Finer-level segmentation for better retrieval precision.

6. **Embed content**  
   Use sentence embedding model (`all-MiniLM-L6-v2`).

7. **Add metadata**  
   Attach `part`, `section`, and `article` as payload.

---

### Qdrant Vector DB

8. **Create `document collection`**  
   Stores all document chunks with embeddings and metadata.

9. **Create `document collection cache`**  
   Stores previously seen query results for reuse.

10. **Embed points**  
    Vectorized chunks are inserted into the main collection.

---

### Semantic Search & Caching

11. **`search(query)`**
    - Checks `cache` collection first.
    - If found → return cached result.
    - If not → search main `document collection`.

12. **Update cache**  
    Stores new query + result in cache for future reuse.

13. **Retrieve `relevant_context`**  
    Search result passed back to main app for response generation.

---

### Re-ranking

14. **Re-rank results**  
    Use a cross-encoder (e.g., `ms-marco-MiniLM`) to refine ranking.

15. **Select top 3**  
    Final `top_3_context` chosen for LLM prompt.


---

### Generation Flow

16. **User sends query**  
    A natural language question is submitted.

17. **Main program dispatches search**  
    Sends query to semantic search module.

18. **Send to OpenAI**  
    Passes `query` + `relevant_context` to OpenAI for generation.

---


<div style="page-break-after: always;"></div>

## Solution Implementation

### Imports

import all the required libraries and modules

In [1]:
from IPython.display import JSON, Markdown, display
import pdfplumber
from sentence_transformers import SentenceTransformer, CrossEncoder
from qdrant_client import QdrantClient
from hashlib import sha256
import pandas as pd
import openai
import json
import ast


### Embedding Layer

#### Text Processing
- **is_useful_page**: Filters out pages that are not relevant.
- **get_part**: Extracts the part from the text.
- **get_section**: Extracts the section from the text.
- **process_document**: Reads the PDF and organizes the text into a structured format.

In [2]:
pdf_document_file_path = "./Life Insurance Policy Sample.pdf"

def is_useful_page(text):
    return "Section" in text.strip().splitlines()[-1]

def get_part(text):
    return text.strip().splitlines()[-2]

def get_section(text):
    last_line = text.strip().splitlines()[-1]
    section_with_page_number = "Section " + last_line.split("Section")[1]
    return section_with_page_number.split("Page")[0].strip().split(",")[0].strip()

def process_document(document_path):
    data = {}

    with pdfplumber.open(document_path) as pdf:
        for page in pdf.pages:
            full_text = page.extract_text()
            if is_useful_page(full_text):
                part                = get_part(full_text)
                section             = get_section(full_text)
    
                if part not in data:
                    data[part] = {}
                data_part = data[part]
    
                if section not in data_part:
                    data_part[section] = []
                data_part_section = data_part[section]
    
                data_part_section.extend(full_text.splitlines())
    return data

# JSON(process_document(pdf_document_file_path))

#### Chunking
- **is_not_same**: Compares two strings to check if they are not the same (ignoring case and spaces).
- **get_chunks**: Processes the structured data to create chunks of text, each associated with a part, section, and article.

In [3]:
def is_not_same(left, right):
    return left.replace(" ", "").lower() not in right.replace(" ", "").lower()

def get_chunks(data):
    documents = []
    for part,part_detail in data.items():
        for section, section_detail in part_detail.items():
            article = ""
            article.replace(" ", "").lower()
            content = ""
            for line in section_detail:
                if is_not_same(part, line) and is_not_same(section, line) and is_not_same("This policy has been updated effective", line):
                    if "Article " in line and " - " in line:
                        if article and content:
                            documents.append({
                                "part": part,
                                "section": section,
                                "article": article,
                                "content": content
                            })
                        article = line
                        content = ""
                        continue
                    else:
                        content += line + " "
                else:
                    continue
    
            # Append the last article and content
            if article and content:
                documents.append({
                    "part": part,
                    "section": section,
                    "article": article,
                    "content": content
                })
    return documents

#### Embedding Logic
- **get_embeddable_points**: Converts the chunks into a format suitable for embedding, including vector representation and metadata.

In [4]:
sentence_embedding_model_name = "all-MiniLM-L6-v2"
sentence_embedding_model = SentenceTransformer(sentence_embedding_model_name)

def get_embeddable_points(chunks):
    points = [{
        "id" : i,
        "vector": sentence_embedding_model.encode(chunk["content"]),
        "payload": {
            "part": chunk["part"],
            "section": chunk["section"],
            "article": chunk["article"],
            "content": chunk["content"],
            "text_length": len(chunk["content"])
        }
    } for i,chunk in enumerate(chunks)]
    return points

#### Prepare Vector DB
- **QdrantClient**: Connects to the Qdrant database.
- **delete_collection**: Deletes any existing collections to start fresh.
- **create_collection**: Creates a new collection for storing the document chunks.

In [ ]:
client = QdrantClient(
    host = 'localhost',
    port = 6333
)

client.delete_collection(
    collection_name = "life_insurance_policy_documents"
)

client.create_collection(
    collection_name = "life_insurance_policy_documents",
    vectors_config = {
            "size": 384,
            "distance": "Cosine"
    }
)

client.delete_collection(
    collection_name = "life_insurance_policy_documents_cache"
)
client.create_collection(
    collection_name = "life_insurance_policy_documents_cache",
    vectors_config = {
            "size": 384,
            "distance": "Cosine"
    }
)

##### output
- **empty_collections.png**: Shows the empty collections in Qdrant.

![empty_collections.png](./empty_collections.png)

#### Embedding the chunks
- **upsert**: Inserts the chunks into the Qdrant collection.

In [ ]:
part_section_data = process_document(pdf_document_file_path)
chunks = get_chunks(part_section_data)
embeddable_points = get_embeddable_points(chunks)

client.upsert(
    collection_name = "life_insurance_policy_documents",
    points = embeddable_points
)

##### output
- **filled_collection.png**: Shows the filled collection in Qdrant.

![filled_collection.png](filled_collection.png)

<div style="page-break-after: always;"></div>

### Search Layer

#### Search & Cache
- **hash_query**: Generates a hash for the query string.
- **save_to_cache**: Saves the query and its results to the cache collection.
- **query_cache_collection**: Queries the cache collection for previously seen queries.
- **query_collection**: Queries the main collection for the given query.
- **search**: Main function that checks the cache first, then queries the main collection if not found.

In [7]:
def hash_query(query: str) -> str:
    return sha256(query.encode()).hexdigest()

def save_to_cache(query: str, points: list) -> None:
    query_embedding = sentence_embedding_model.encode(query)
    query_hash = hash_query(query)
    client.upsert(
        collection_name = "life_insurance_policy_documents_cache",
        points = [{
            "id": int(query_hash[:16], 16),
            "vector": query_embedding,
            "payload": {
                "query": query,
                "query_hash": query_hash,
                "points": points
            }
        }]
    )

def query_cache_collection(query: str, limit=5) -> list:
    query_hash = hash_query(query)
    query_response = client.query_points(
        collection_name = "life_insurance_policy_documents_cache",
        query_filter = {
            "must" : [
                {"key": "query_hash", "match": {"value": query_hash}},
            ]
        },
        limit = limit,
        with_payload = True
    )
    points = []
    if len(query_response.points) > 0:
        points = query_response.points[0].payload['points']
    return points

def query_collection(query: str, limit=5) -> list:
    query_response = client.query_points(
        collection_name = "life_insurance_policy_documents",
        query = sentence_embedding_model.encode(query),
        limit = limit,
        with_payload = True
    )
    results = []
    [results.append({
        "id" : point.id,
        "version": point.version,
        "score": point.score,
        "part": point.payload["part"],
        "section": point.payload["section"],
        "article": point.payload["article"],
        "content": point.payload["content"],
        "text_length": point.payload["text_length"]
    }) for point in query_response.points]
    return results

def search(query: str, limit=3) -> list:
    points = query_cache_collection(query, limit)
    if len(points) == 0:
        points = query_collection(query, limit)
        save_to_cache(query, points)
        print("Cache miss!")
    else:
        print("Cache hit!")
    return points

<div style="page-break-after: always;"></div>

##### search test
- **search**: Searches for the query in the Qdrant collection.
- Displays a `Cache miss!`

In [8]:
semantic_search_result = search("who has the authority to change the policy?")

Cache miss!


In [9]:
formatted_json = json.dumps(semantic_search_result, indent=2)
md = f"```json\n{formatted_json}\n```"
display(Markdown(md))

```json
[
  {
    "id": 1,
    "version": 0,
    "score": 0.5939084,
    "part": "PART II - POLICY ADMINISTRATION",
    "section": "Section  A - Contract",
    "article": "Article 2 - Policy Changes",
    "content": "Insurance under this Group Policy runs annually to the Policy Anniversary, unless sooner terminated. No agent, employee, or person other than an officer of The Principal has authority to change this Group Policy, and, to be effective, all such changes must be in Writing and Signed by an officer of The Principal. The Principal reserves the right to change this Group Policy as follows: a. Any or all provisions of this Group Policy may be amended or changed at any time, including retroactive changes, to the extent necessary to meet the requirements of any law or any regulation issued by any governmental agency to which this Group Policy is subject. b. Any or all provisions of this Group Policy may be amended or changed at any time when The Principal determines that such amendment is required for consistent application of policy provisions. c. By Written agreement between The Principal and the Policyholder, this Group Policy may be amended or changed at any time as to any of its provisions. Any change to this Group Policy, including, but not limited to, those in regard to coverage, benefits, and participation privileges, may be made without the consent of any Member or Dependent. Payment of premium beyond the effective date of the change constitutes the Policyholder's consent to the change. ",
    "text_length": 1308
  },
  {
    "id": 9,
    "version": 0,
    "score": 0.47650385,
    "part": "PART II - POLICY ADMINISTRATION",
    "section": "Section  A - Contract",
    "article": "Article 10 - Policy Interpretation",
    "content": "T he Principal has complete discretion to construe or interpret the provisions of this group insurance policy, to determine eligibility for benefits, and to determine the type and extent of benefits, if any, to be provided. The decisions of The Principal in such matters shall be controlling, binding and final as between The Principal and persons covered by this Group Policy, subject to the Claims Procedures in PART IV, Section D. ",
    "text_length": 434
  },
  {
    "id": 5,
    "version": 0,
    "score": 0.46555924,
    "part": "PART II - POLICY ADMINISTRATION",
    "section": "Section  A - Contract",
    "article": "Article 6 - Information to be Furnished",
    "content": "The Policyholder must, upon request, give The Principal all information needed to administer this Group Policy. If a clerical error is found in this information, The Principal may at any time adjust premium to reflect the facts. An error will not invalidate insurance that would otherwise be in force. Neither will an error continue insurance that would otherwise be terminated. The Principal may inspect, at any reasonable time, all Policyholder records, which relate to this Group Policy. ",
    "text_length": 491
  }
]
```

<div style="page-break-after: always;"></div>

##### cache test
- **search**: Searches for the query in the Qdrant collection.
- Displays a `Cache hit!`

In [10]:
semantic_search_result = search("who has the authority to change the policy?")

Cache hit!


In [11]:
formatted_json = json.dumps(semantic_search_result, indent=2)
md = f"```json\n{formatted_json}\n```"
display(Markdown(md))

```json
[
  {
    "id": 1,
    "version": 0,
    "score": 0.5939084,
    "part": "PART II - POLICY ADMINISTRATION",
    "section": "Section  A - Contract",
    "article": "Article 2 - Policy Changes",
    "content": "Insurance under this Group Policy runs annually to the Policy Anniversary, unless sooner terminated. No agent, employee, or person other than an officer of The Principal has authority to change this Group Policy, and, to be effective, all such changes must be in Writing and Signed by an officer of The Principal. The Principal reserves the right to change this Group Policy as follows: a. Any or all provisions of this Group Policy may be amended or changed at any time, including retroactive changes, to the extent necessary to meet the requirements of any law or any regulation issued by any governmental agency to which this Group Policy is subject. b. Any or all provisions of this Group Policy may be amended or changed at any time when The Principal determines that such amendment is required for consistent application of policy provisions. c. By Written agreement between The Principal and the Policyholder, this Group Policy may be amended or changed at any time as to any of its provisions. Any change to this Group Policy, including, but not limited to, those in regard to coverage, benefits, and participation privileges, may be made without the consent of any Member or Dependent. Payment of premium beyond the effective date of the change constitutes the Policyholder's consent to the change. ",
    "text_length": 1308
  },
  {
    "id": 9,
    "version": 0,
    "score": 0.47650385,
    "part": "PART II - POLICY ADMINISTRATION",
    "section": "Section  A - Contract",
    "article": "Article 10 - Policy Interpretation",
    "content": "T he Principal has complete discretion to construe or interpret the provisions of this group insurance policy, to determine eligibility for benefits, and to determine the type and extent of benefits, if any, to be provided. The decisions of The Principal in such matters shall be controlling, binding and final as between The Principal and persons covered by this Group Policy, subject to the Claims Procedures in PART IV, Section D. ",
    "text_length": 434
  },
  {
    "id": 5,
    "version": 0,
    "score": 0.46555924,
    "part": "PART II - POLICY ADMINISTRATION",
    "section": "Section  A - Contract",
    "article": "Article 6 - Information to be Furnished",
    "content": "The Policyholder must, upon request, give The Principal all information needed to administer this Group Policy. If a clerical error is found in this information, The Principal may at any time adjust premium to reflect the facts. An error will not invalidate insurance that would otherwise be in force. Neither will an error continue insurance that would otherwise be terminated. The Principal may inspect, at any reasonable time, all Policyholder records, which relate to this Group Policy. ",
    "text_length": 491
  }
]
```

###### output
- **cache_test_output.png**: Shows that the queries are cached

![cache_test_output.png](cache_test_output.png)

<div style="page-break-after: always;"></div>

#### Re-Ranking
- **get_df_from_points**: Converts the list of points into a DataFrame for easier manipulation.
- **get_re_ranked_results**: Uses a cross-encoder model to re-rank the results based on their relevance to the query.
- **search_with_re_ranking**: Main function that performs the search and re-ranking.

In [12]:
def get_df_from_points(vector_points):
    flat_items = []
    [flat_items.append({
        "id" : point["id"],
        "version": point["version"],
        "score": point["score"],
        "part": point["part"],
        "section": point["section"],
        "article": point["article"],
        "content": point["content"],
        "text_length": point["text_length"]
    }) for point in vector_points]

    return pd.DataFrame(flat_items)

def get_re_ranked_results(query, vector_points):
    re_ranking_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    results_df = get_df_from_points(vector_points)
    results_df['re_ranking_scores'] = results_df.apply(lambda x: re_ranking_model.predict([[query, x['content']]])[0], axis = 1)
    return results_df

def search_with_re_ranking(query, limit=5):
    vector_points = search(query, limit)
    results = get_re_ranked_results(query, vector_points)
    return results[['part', 'section', 'article', 'content']]

##### re-ranking comparison with distance score
- **search**: Searches for the query in the Qdrant collection.
- **get_re_ranked_results**: Re-ranks the results based on their relevance to the query.
- **similaritysearch_reranking_comparision_df**: DataFrame showing the comparison between the original scores and the re-ranking scores.

In [13]:
query = "who has the authority of changing the policy?"
vector_points = search(query)
re_ranked_results = get_re_ranked_results(query, vector_points)
similaritysearch_reranking_comparision_df = re_ranked_results.sort_values(by = ['re_ranking_scores'], ascending = False)
similaritysearch_reranking_comparision_df = similaritysearch_reranking_comparision_df[['score', 're_ranking_scores', 'id']]
similaritysearch_reranking_comparision_df

Cache miss!


,score,re_ranking_scores,id
0,0.548356,4.623034,1
2,0.427931,-5.191046,5
1,0.448610,-6.051162,9


##### observation
- re-ranking produces a different result than regular semantic search
- based on regular semantic search, the chosen top 3 chunks are 1, 9 and 5
- based on re-ranking, the chosen top 3 chunks are 1, 5 and 9

<div style="page-break-after: always;"></div>

### Generation Layer

#### openai connection and initial system prompt
- **openai**: Connects to the OpenAI API.
- **get_chat_model_completions**: Sends the messages to the OpenAI API and retrieves the response.
- **get_insurance_answers**: Prepares the system prompt and user question, and sends it to the OpenAI API for generation.
- **user_query**: Main function that takes the user query, performs the search, and generates the response.

In [14]:
with open('../OPENAI_API_KEY.txt', 'r') as openai_key_file:
    openai.api_key = openai_key_file.readline()

def get_chat_model_completions(messages):
    response_llm = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        temperature = 0,
        messages = messages
    )
    return response_llm.choices[0].message.content

def get_insurance_answers(question, relevant_context):

    delimiter = "#"*10
    
    system_prompt = f"""
        You are a helpful assistant in the insurance domain who understand the policies and documents related to the insurance domain. Specifically life insurance.
        You will help the users by answering their important questions.
        You will be given relevant context to answer the user's questions. This relevant context is created from an insurance document. The relevant context will be given to you in the format of a dataframe.
        The relevant context contains "part" which represents one of the main parts of the insurance document.
        The relevant context also contains "section" which represents one of the sections under a part in the insurance document.
        The relevant context also contains "article" which represents an article under a part and section, in the insurance document.
        The relevant context also contains "content" which belongs to a part/section/article.
        Each relevant context in the dataframe is an important piece of information. The rows in the dataframe are sorted based on their relative importance. The first document is most relevant and so on.

        {delimiter}

        Your job is to provide the answer of the question "{question}" by using the relevant context available in {relevant_context}. If you are not sure of the answer, say that you do not know.
        

        {delimiter}

        Follow these guidelines when performing the task.
        1. You don't have to necessarily use all the information in the dataframe. Use only information which is relevant.
        2. Use the "part", "section", "article" and the summary of "content" column in the dataframe for any citations.
        3. You can use your own knowledge to explain specific terms in the domain of life insurance.

        {delimiter}

        The output should follow the below guidelines:
        1. The output should contain a final answer in the manner user has asked, relevant part/section/article/content-summary in a tabular format.
        2. The output should answer the question directly addressing the user and avoid any additional information. If you think that the provided relevant context is not related to the question, mention it in the response.
        
    """
    
    messages = [
                {
                    "role": "system", "content": system_prompt
                },
                {
                    "role": "user", "content": question
                }
              ]
    return get_chat_model_completions(messages)

def user_query(query:str) -> tuple:
    print(query)
    search_results_df = search_with_re_ranking(query)
    search_results_df = search_results_df[:3]
    semantic_search_output = json.dumps(search_results_df[['part', 'section', 'article']].to_dict(orient='records'), indent=2)
    llm_output = get_insurance_answers(query, search_results_df)
    return (semantic_search_output, llm_output)



<div style="page-break-after: always;"></div>

## Query Search

### First Query

In [15]:
semantic_search_output, llm_output = user_query("Who is allowed to make changes to this group policy and how are those changes validated?")

Who is allowed to make changes to this group policy and how are those changes validated?
Cache miss!


#### search

In [16]:
data = ast.literal_eval(semantic_search_output)
formatted_json = json.dumps(data, indent=2)
md = f"```json\n{formatted_json}\n```"
display(Markdown(md))

```json
[
  {
    "part": "PART II - POLICY ADMINISTRATION",
    "section": "Section  A - Contract",
    "article": "Article 2 - Policy Changes"
  },
  {
    "part": "PART II - POLICY ADMINISTRATION",
    "section": "Section  A - Contract",
    "article": "Article 10 - Policy Interpretation"
  },
  {
    "part": "PART II - POLICY ADMINISTRATION",
    "section": "Section  A - Contract",
    "article": "Article 6 - Information to be Furnished"
  }
]
```

#### generation

In [17]:
display(Markdown(llm_output))

The Principal has complete discretion to consent to any changes to the group policy. Changes are validated through the Principal's authority as outlined in the policy.

| Part                                | Section                     | Article                          | Content Summary                                                                 |
|-------------------------------------|-----------------------------|----------------------------------|---------------------------------------------------------------------------------|
| PART II - POLICY ADMINISTRATION     | Section A - Contract        | Article 2 - Policy Changes       | The Principal has complete discretion to consent to any changes to the policy.   |

<div style="page-break-after: always;"></div>

### Second Query

In [18]:
semantic_search_output, llm_output = user_query("When does my dependent life insurance coverage begin and are there any conditions?")

When does my dependent life insurance coverage begin and are there any conditions?
Cache miss!


#### search

In [19]:
data = ast.literal_eval(semantic_search_output)
formatted_json = json.dumps(data, indent=2)
md = f"```json\n{formatted_json}\n```"
display(Markdown(md))

```json
[
  {
    "part": "PART III - INDIVIDUAL REQUIREMENTS AND RIGHTS",
    "section": "Section  A - Eligibility",
    "article": "Article 3 - Dependent Life Insurance"
  },
  {
    "part": "PART III - INDIVIDUAL REQUIREMENTS AND RIGHTS",
    "section": "Section  B - Effective Dates",
    "article": "Article 3 - Dependent Life Insurance"
  },
  {
    "part": "PART III - INDIVIDUAL REQUIREMENTS AND RIGHTS",
    "section": "Section  C - Individual Terminations",
    "article": "Article 3 - Dependent Life Insurance"
  }
]
```

#### generation

In [20]:
display(Markdown(llm_output))

Your dependent life insurance coverage begins when you are eligible for it, which is available only with your own life insurance coverage. Additionally, there are conditions related to the effective dates of the insurance.

Here is the relevant information:

| Part                                         | Section                               | Article                          | Content Summary                                                                                     |
|----------------------------------------------|---------------------------------------|----------------------------------|-----------------------------------------------------------------------------------------------------|
| PART III - INDIVIDUAL REQUIREMENTS AND RIGHTS | Section A - Eligibility               | Article 3 - Dependent Life Insurance | A person will be eligible for Dependent Life Insurance only if they have their own life insurance coverage. |
| PART III - INDIVIDUAL REQUIREMENTS AND RIGHTS | Section B - Effective Dates           | Article 3 - Dependent Life Insurance | Dependent Life Insurance is available only with your own life insurance coverage.                   |

In summary, your dependent life insurance coverage begins when you have your own life insurance, and it is subject to the conditions outlined in the policy.

<div style="page-break-after: always;"></div>

### Third Query

#### query

In [21]:
semantic_search_output, llm_output = user_query("What happens to my life insurance benefit if I become terminally ill?")

What happens to my life insurance benefit if I become terminally ill?
Cache miss!


#### search

In [22]:
data = ast.literal_eval(semantic_search_output)
formatted_json = json.dumps(data, indent=2)
md = f"```json\n{formatted_json}\n```"
display(Markdown(md))

```json
[
  {
    "part": "PART IV - BENEFITS",
    "section": "Section  A - Member Life Insurance",
    "article": "Article 7 - Accelerated Benefits"
  },
  {
    "part": "PART IV - BENEFITS",
    "section": "Section  C - Dependent Life Insurance",
    "article": "Article 2 - Death Benefits Payable"
  },
  {
    "part": "PART IV - BENEFITS",
    "section": "Section  A - Member Life Insurance",
    "article": "Article 2 - Death Benefits Payable"
  }
]
```

#### generation

In [23]:
display(Markdown(llm_output))

If you become terminally ill, you may be eligible for accelerated benefits under your life insurance policy. This means that you can receive a portion of your life insurance benefit while you are still alive, which can help cover medical expenses or other costs associated with your illness.

Here is the relevant information:

| Part                     | Section                          | Article                     | Content Summary                                                                                     |
|--------------------------|----------------------------------|-----------------------------|-----------------------------------------------------------------------------------------------------|
| PART IV - BENEFITS       | Section A - Member Life Insurance | Article 7 - Accelerated Benefits | You may qualify for accelerated benefits if you become terminally ill, allowing you to access part of your life insurance benefit early. |

This information indicates that you can access your life insurance benefit if you are diagnosed with a terminal illness.

<div style="page-break-after: always;"></div>

# Conclusions

## Challenges faced

- Identifying the right chunking strategy for the document.
- Understanding the api for Qdrant and how to use it.
- Caching the points changes the data structure from ScoredPoints to dict.